In [ ]:
# GAN은 지금까지 하나의 넷웤이면 얜 두개의 넷웤(GENERATIVE, ADVERSARIAL)
# 속이려는자와 감별하려는자의 싸움(MIN과 MAX의 치열한 피튀기는 싸움)
# 대표적인 UNSUPERVISED LEARNING
# 0.5가 나오면 완벽한 제너레이터

# DISCRIMINATOR 는 진짜일수록 1에 가까운숫자를 내밈
#  GENERATOR는 가짜를 만드는 놈 (실제 데이터분포와 유사한 데이터분포를 GENERATE하는것)

# D의 목적은 진짜를 1, 가짜를 0으로 잘 구분했을떄 D가 최대화 된다
# G의 목적은 가짜를 1로 구분했을떄, G가 수학적으로 최소화 된다

# COST의 최저치는 Log4가 가장 학습이 잘된 상태(수학적으로 그럼)

#실제 분포를 복제함으로써 시뮬레이션할때 좋다

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
num_steps =10000
batch_size =128
learning_rate= 0.0002
# G, D - DNN (256개)
image_dim =784
gen_hidden_dim=256
disc_hidden_dim = 256
noise_dim = 100

In [9]:
def he_init(shape):
    return tf.random_normal(shape=shape, stddev=1 / tf.sqrt(shape[0]/2))

weights = {
    'gen_hidden1': tf.Variable(he_init([noise_dim, gen_hidden_dim])),
    'gen_out' : tf.Variable(he_init([gen_hidden_dim, image_dim])),
    'disc_hidden1' : tf.Variable(he_init([image_dim, disc_hidden_dim])),
    'disc_out' : tf.Variable(he_init([disc_hidden_dim,1]))
}

biases = {
    'gen_hidden1' : tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc_hidden1' : tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out' : tf.Variable(tf.zeros([1]))
}

In [14]:
#망은 cnn을 중간에 껴도되고 시그모이드나 렐루 자유롭게 쓸 수있따. rnn은 안끼는 편
def generator(x): #G 의 nn 구성
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer, weights['gen_out'])
    output_layer = tf.add(output_layer, biases['gen_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer

def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer, weights['disc_out'])
    output_layer = tf.add(output_layer, biases['disc_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer
    

In [15]:
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name = 'input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, image_dim], name = 'disc_noise')

gen_sample = generator(gen_input)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample)

gen_loss = -tf.reduce_mean(tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real)+ tf.log(1-disc_fake)) 
#maximize는 -붙인담에 minimize한것과 동일

optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

gen_vars = [weights['gen_hidden1'], weights['gen_out'],
           biases['gen_hidden1'], biases['gen_out']]
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
           biases['disc_hidden1'], biases['disc_out']]

train_gen = optimizer_gen.minimize(gen_loss, var_list = gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list = disc_vars) 
#두개를 분리 시키기위해서 따로 묶어야 한다
#동시에 두개의 네트워크를 학습시켜야되니까 나눠야 하는 번거로움쓰

#KL DIVERGENCE 를 통해 LOG4 이후에 COST감소를 확인할 수 있다 D,G가 얼마나 비슷한가를 확인

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(num_steps):
    batch_xs, _ = mnist.train.next_batch(batch_size) # disc의 인풋
    z = np.random.uniform(-1.,1.,size=[batch_size, noise_dim]) # gen 인풋
    _, _, gl, dl = sess.run([train_gen,train_disc,gen_loss, disc_loss],
                        feed_dict={disc_input:batch_xs, gen_input:z})
    if i % 100 == 0:
        print("Step:",i,"G Loss:",gl,"D Loss:",dl)
# Margin Maximize Problem


Step: 0 G Loss: 0.66084725 D Loss: 1.4737036
Step: 100 G Loss: 4.3877916 D Loss: 0.058852132
Step: 200 G Loss: 3.1181262 D Loss: 0.22846541
Step: 300 G Loss: 2.5478635 D Loss: 0.30368263
Step: 400 G Loss: 2.7490153 D Loss: 0.20249903
Step: 500 G Loss: 3.3330235 D Loss: 0.10782687
Step: 600 G Loss: 3.5284529 D Loss: 0.068967596
Step: 700 G Loss: 3.7051613 D Loss: 0.07452514
Step: 800 G Loss: 3.615132 D Loss: 0.05998452
Step: 900 G Loss: 3.865469 D Loss: 0.050592747
Step: 1000 G Loss: 3.733379 D Loss: 0.06774552
Step: 1100 G Loss: 3.5877113 D Loss: 0.06514835
Step: 1200 G Loss: 3.9382575 D Loss: 0.047725476
Step: 1300 G Loss: 3.6427078 D Loss: 0.072430156
Step: 1400 G Loss: 3.817533 D Loss: 0.05119943
Step: 1500 G Loss: 3.785926 D Loss: 0.06998883
Step: 1600 G Loss: 4.153843 D Loss: 0.055082127
Step: 1700 G Loss: 4.2033453 D Loss: 0.06917495
Step: 1800 G Loss: 4.017217 D Loss: 0.051693022
Step: 1900 G Loss: 4.402749 D Loss: 0.059229188
Step: 2000 G Loss: 4.442643 D Loss: 0.051732853
Step

In [ ]:
#GAN 은 DCGAN을 많이 씀 (https://dev-strender.github.io/articles/2017-07/decan-introduction)